In [22]:
import leafmap.foliumap as leafmap
import geopandas
import os, sys
import pickle
import pandas as pd

In [11]:
sys.path.append('CONFIG_files/')

In [12]:
import CONFIG_eLLOiWHFRq as CONFIG

CONFIG.REGION

'COMBINED'

In [13]:
run_prefix = 'eLLOiWHFRq'

with open(f'/data/hpcdata/users/marron31/_experiments/{run_prefix}/predictions.pkl', 'rb') as handle:
    predictions = pickle.load(handle)

In [14]:
def delete_uncomplete_runs_from_predictions(predictions, CONFIG):
    for k,run in predictions.items():
#         print(k, run.keys())
#         print(len(CONFIG.params['k']))
        if len(run.keys()) < len(CONFIG.params['k']):
            del run
            print(f'deleted run {k}')

In [15]:
delete_uncomplete_runs_from_predictions(predictions, CONFIG)

In [16]:
def create_k_all_predictions(predictions, CONFIG):
    # Create predictions for k_all
    for run in predictions.keys():
        for i in range(len(CONFIG.params['k'])):
            predictions[run][f'k{i}']['k_fold'] = i
            if i == 0:
                predictions[run]['k_all'] = predictions[run][f'k{i}']
            else:
                predictions[run]['k_all'] = pd.concat([predictions[run]['k_all'],predictions[run][f'k{i}']])

In [19]:
create_k_all_predictions(predictions, CONFIG)

In [60]:
df_stations = predictions['MLP_[10]_bgmm_B=128_D=0']['k_all'].groupby('Station').agg('first')[['k_fold','X','Y']].reset_index()

In [61]:
gdf = geopandas.GeoDataFrame(
    df_stations, geometry=geopandas.points_from_xy(df_stations.X, df_stations.Y, crs='EPSG:4326'))

In [73]:
m = leafmap.Map(center=(40, -100), zoom=4)
m.add_gdf(gdf,zoom_to_layer=True)
m.add_points_from_xy(df_stations, 'X', 'Y', color_column='k_fold', layer_name='Stations')

The provided color (lightred) is invalid. Using the default black color.
'#lightred' is not in web format. Need 3 or 6 hex digit.


In [74]:
m